In [1]:
# Student asks question about physics
# decide if beginner or advancd question
# INPUT -> ROUTER --> LLM decides Chain -> Chain -> Output

In [2]:
beginner_template="""You are a physics teacher who is really focused on beginners, explaining concepts simply. You assume no prior knowledge.
Here is your question: \n
{input}
"""

In [3]:
expert_template="""You are a physics professor who explains physics topics to advanced audience members. You can assume anyone you answer to has a PHD in Physics.
here is your question: \n
{input}"""

In [4]:
# ROUTE PROMPT INFORMATION
# [] NAME, DESCRIPTION, TEMPLATE

In [5]:
prompt_infos = [
    {
        'name': "Beginner Physics",
        'description': "Answers basic physics questions",
        'template': beginner_template,
    },
    {
        'name': "Expert Physics",
        'description': "Answers advanced physics questions",
        'template': expert_template,
    },
]

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = ChatOpenAI()
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [8]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [9]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [10]:
destinations = [ f"{p['name']}: {p['description']}" for p in prompt_infos]

In [11]:
destination_str = "\n".join(destinations)

In [12]:
print(destination_str)

Beginner Physics: Answers basic physics questions
Expert Physics: Answers advanced physics questions


In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [14]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

In [15]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Beginner Physics: Answers basic physics questions
Expert Physics: Answers advanced physics questions


In [16]:
router_prompt = PromptTemplate(template=router_template, input_variables=['input'], output_parser=RouterOutputParser())

In [17]:
from langchain.chains.router import MultiPromptChain

In [18]:
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [19]:
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

In [20]:
chain.run("How do magnets work")



> Entering new MultiPromptChain chain...


/Users/pete/.local/share/virtualenvs/langchain-udemy-notebooks-Raz3oF7P/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Beginner Physics: {'input': 'How do magnets work'}
> Finished chain.


'Magnets are fascinating objects that have the ability to attract or repel certain materials. The way magnets work is based on the concept of magnetic fields. \n\nEvery magnet has two ends, which we call the North pole and the South pole. These poles are like the opposite ends of a magnet. When you bring two magnets closer to each other, the opposite poles attract each other, while the same poles repel each other. This is why magnets can stick together or push each other away. \n\nBut you might wonder, how does this happen? Well, magnets create something called a magnetic field around them. Think of it as an invisible force field that extends outwards. When another material, like a piece of iron or another magnet, comes into this magnetic field, it feels the force and gets attracted or repelled. \n\nTo explain it further, inside every magnet, there are tiny particles called electrons. These electrons are constantly moving around the atoms that make up the magnet. They have a property c

In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon